In [1]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()

In [2]:
exp = Experiment(workspace=ws, name="udacity-project")

run = exp.start_logging()

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
cpu_cluster_name = "UdacityProject"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [26]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
  
# Specify parameter sampler
ps = RandomParameterSampling({
        "number_of_hidden_layers": choice(range(1,5)),
        "batch_size": choice(16, 32, 64, 128)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

if "training" not in os.listdir():
    os.mkdir("./training")
    
est = SKLearn(source_directory='.', 
   compute_target=cpu_cluster,
   entry_script='train.py',
#             pip_packages=pip_packages,
   conda_packages=['scikit-learn==0.22.2.post1'])





In [20]:
# %%writefile conda_dependencies.yml

# dependencies:
# - python=3.6.2
# - scikit-learn
# - pip:
#   - azureml-defaults

In [21]:
# from azureml.core import Environment

# sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')


# from azureml.core import ScriptRunConfig

# src = ScriptRunConfig(source_directory='.',
#                       script='train.py',
#                       arguments=['--kernel', 'linear', '--penalty', 1.0],
#                       compute_target=cpu_cluster,
#                       environment=sklearn_env)

In [27]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
#                            run_config=src,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

In [28]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)

In [30]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [30]:
assert(hyperdrive_run.get_status() == "Completed")

In [16]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

In [ ]:
import joblib
# Get your best run and save the model from that run.

In [17]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

In [18]:
parameter_value = best_run.get_details()['runDefinition']['arguments']

## AutoML

In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
autods = TabularDatasetFactory.from_delimited_files(path = path)

In [7]:
from train import clean_data

In [11]:
ds = clean_data(autods)

In [9]:
# from train import clean_data

# import pandas as pd
# def clean_data(data):
#     # Dict for cleaning data
#     months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
#     weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

#     # Clean and one hot encode data
#     x_df = data.to_pandas_dataframe().dropna()
#     jobs = pd.get_dummies(x_df.job, prefix="job")
#     x_df.drop("job", inplace=True, axis=1)
#     x_df = x_df.join(jobs)
#     x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
#     x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
#     x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
#     x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
#     contact = pd.get_dummies(x_df.contact, prefix="contact")
#     x_df.drop("contact", inplace=True, axis=1)
#     x_df = x_df.join(contact)
#     education = pd.get_dummies(x_df.education, prefix="education")
#     x_df.drop("education", inplace=True, axis=1)
#     x_df = x_df.join(education)
#     x_df["month"] = x_df.month.map(months)
#     x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
#     x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

#     x_df['y_cleaned'] = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)
    
#     return x_df

# # Use the clean_data function to clean your data.
# ds = clean_data(autods)

NameError: name 'autods' is not defined

In [12]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(ds, test_size=0.2, random_state=11)
label_column_name = "y_cleaned"

In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_settings = {
    "n_cross_validations": 5,
    "primary_metric": "average_precision_score_weighted",
    "verbosity": logging.INFO,
    "enable_stack_ensemble": False,
}

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    debug_log="automl_errors.log",
    training_data=train,
    label_column_name=label_column_name,
    **automl_settings)

In [15]:
# Submit your automl run

run = exp.submit(config = automl_config, show_output = True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_a669213f-f7b2-4c2d-9092-eca3d008bbea,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias toward

In [16]:
# Retrieve and save your best automl model.

best_run, best_model = run.get_output()
best_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=1.4583333333333335, reg_lambda=1.4583333333333335, subsample=1, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.15384615384615385, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.15384615384615385, 0.15384615384615385, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693]))],
         verbose=False)

In [18]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_a669213f-f7b2-4c2d-9092-eca3d008bbea_45,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
delete_compute(cpu_cluster)